In [3]:
import gensim

model = gensim.models.Word2Vec.load("../latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [5]:
import MeCab
import re

mecab = MeCab.Tagger ()
pattern = re.compile('[a-zA-Z]')
noun_list = []

i = 0

for text in model.wv.index2word:
    #print("text", text)
        
    mecab.parse('')#文字列がGCされるのを防ぐ
    node = mecab.parseToNode(text)
    
    b = re.search(pattern,text)  
    #print("result", result)
        
    if b: 
        #print("英語")
        #print(result.group())
        continue 
    
    i +=1
    
    while node:
        #単語を取得
        word = node.surface
        
        #品詞を取得
        pos = node.feature.split(",")[1]
        #print('{0} , {1}'.format(word, pos))
        
        #print("pos", pos)
        #if pos == "一般" or pos == "固有名詞":
        if pos == "一般":
            noun_list.append(word)
        
        #次の単語に進める
        node = node.next
        #print("while")

In [6]:
len(noun_list)

176508

In [7]:
len(set(noun_list))

43413

In [8]:
import pickle

f = open('ippann_noun_list.binaryfile','wb')
noun_list = list(set(noun_list))
pickle.dump(noun_list,f)
f.close

<function BufferedWriter.close>

# 単語ベクトルを辞書で取得

In [23]:
import pickle

f = open('ippann_noun_list.binaryfile','rb')
noun_list = pickle.load(f)

In [24]:
#ランダムで取得

import random
n = 1

num = int(len(noun_list) / n) #半分取得
noun_list = random.sample(noun_list, num)

In [25]:
len(noun_list)

43413

In [26]:
noun_dic = {}

for word in noun_list:
    try:
        noun_dic[word] = model.wv[word]
    except:
        continue

In [27]:
len(noun_dic)

40853

In [28]:
import pickle

f = open('ippann_noun_dic.binaryfile','wb')
#f = open('koyuu_noun_sample_dic.binaryfile','wb')
pickle.dump(noun_dic,f)
f.close

<function BufferedWriter.close>

# 単語ベクトル集合を算出

In [33]:
import pickle

f = open('ippann_noun_dic.binaryfile','rb')
noun_dic= pickle.load(f)

In [34]:
import numpy as np

noun_vec_list = list(noun_dic.values())

re_noun_vec_list = []

In [35]:
import random 
n = 1

sample_list = random.sample(noun_vec_list, int(len(noun_vec_list)/n))

In [36]:
len(sample_list)

40853

In [37]:
for i in sample_list:
    a = i.reshape(50,)
    re_noun_vec_list.append(a)

re_noun_vec_list = np.array(re_noun_vec_list)

In [38]:
len(re_noun_vec_list)

40853

In [39]:
import pickle

f = open('sample_ippann_noun_vec_list.binaryfile','wb')
#f = open('koyuu_noun_sample_dic.binaryfile','wb')
pickle.dump(re_noun_vec_list, f)
f.close

<function BufferedWriter.close>

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

eps_minPts = {}
minPts_data = {}

print("noun_vec_list数 : ", len(re_noun_vec_list))

for eps in range(5,50,5):
    eps = eps/10
    
    for minPts in range(1,100, 10):
        
        dbscan = DBSCAN(eps=eps,min_samples=minPts).fit(re_noun_vec_list)
        
        #print(dbscan)
        
        y_dbscan = dbscan.labels_
        
        print("y_dbscan", y_dbscan)
        
        print("eps:",eps,",minPts:", minPts)
        
        # outlier数
        print("outlier数", len(np.where(y_dbscan ==-1)[0]))
        
        # クラスタ数
        print("クラスタ数", np.max(y_dbscan) +2)
        
        # クラスタ1に含まれる点の数
        print("クラスタ1に含まれる点の数", len(np.where(y_dbscan ==0)[0]))
        
        # クラスタ2に含まれる点の数
        print("クラスタ2に含まれる点の数", len(np.where(y_dbscan ==1)[0]))
        
        # クラスタ3に含まれる点の数
        print("クラスタ3に含まれる点の数", len(np.where(y_dbscan ==2)[0]))
        
        print("                            ")
        
        minPts_data[minPts] = y_dbscan
        eps_minPts[eps] = minPts_data
    print(eps) 
    break

In [ ]:
import pickle

f = open('dbscan_dic.binaryfile','wb')
pickle.dump(eps_minPts,f)
f.close